A wall clock falls down and breaks into exactly two pieces.The addition of the numbers in the smaller piece is 31.
And the numbers in the bigger piece addup to 30.How exactly did the wall clock break? 

In [2]:
import copy

In [50]:
def generate_set_partitions(ns, m):
    '''
    Find all possible m subset partitions of set ns
    ns: input list 
    m: number of partitions
    returns: generator that produces a tuple (p1, p2, ...pm) of each subset.
    Acknwoledgement: Adeel Zafar Soomro (https://codereview.stackexchange.com/a/1944) based on 
                    Knuth in the Art of Computer Programming, Volume 4, Fascicle 3B
    '''
    def visit(n, a):
        ps = [[] for i in range(m)]
        for j in range(n):
            ps[a[j + 1]].append(ns[j])
        return ps

    def f(mu, nu, sigma, n, a):
        if mu == 2:
            yield visit(n, a)
        else:
            for v in f(mu - 1, nu - 1, (mu + sigma) % 2, n, a):
                yield v
        if nu == mu + 1:
            a[mu] = mu - 1
            yield visit(n, a)
            while a[nu] > 0:
                a[nu] = a[nu] - 1
                yield visit(n, a)
        elif nu > mu + 1:
            if (mu + sigma) % 2 == 1:
                a[nu - 1] = mu - 1
            else:
                a[mu] = mu - 1
            if (a[nu] + sigma) % 2 == 1:
                for v in b(mu, nu - 1, 0, n, a):
                    yield v
            else:
                for v in f(mu, nu - 1, 0, n, a):
                    yield v
            while a[nu] > 0:
                a[nu] = a[nu] - 1
                if (a[nu] + sigma) % 2 == 1:
                    for v in b(mu, nu - 1, 0, n, a):
                        yield v
                else:
                    for v in f(mu, nu - 1, 0, n, a):
                        yield v

    def b(mu, nu, sigma, n, a):
        if nu == mu + 1:
            while a[nu] < mu - 1:
                yield visit(n, a)
                a[nu] = a[nu] + 1
            yield visit(n, a)
            a[mu] = 0
        elif nu > mu + 1:
            if (a[nu] + sigma) % 2 == 1:
                for v in f(mu, nu - 1, 0, n, a):
                    yield v
            else:
                for v in b(mu, nu - 1, 0, n, a):
                    yield v
            while a[nu] < mu - 1:
                a[nu] = a[nu] + 1
                if (a[nu] + sigma) % 2 == 1:
                    for v in f(mu, nu - 1, 0, n, a):
                        yield v
                else:
                    for v in b(mu, nu - 1, 0, n, a):
                        yield v
            if (mu + sigma) % 2 == 1:
                a[nu - 1] = 0
            else:
                a[mu] = 0
        if mu == 2:
            yield visit(n, a)
        else:
            for v in b(mu - 1, nu - 1, (mu + sigma) % 2, n, a):
                yield v

    n = len(ns)
    a = [0] * (n + 1)
    for j in range(1, m + 1):
        a[n - m + j] = j - 1
    return f(m, n, 0, n, a)

The sum of 1+2+...12 = 78. So its not possible for a splitting the clock to proudct (30, 31) since total would be 61. This means, some of the numbers must be altered due to split. The possibilities are: two digit numbers (10, 11, 12) are split e.g 12 becomes 1 and 2 and goes into separate parts.  Even with this there is not mathematical partition that gives sums of 30, 31.    eg. changing 12 to 1 and 2, removes 12 adds (1+3) i.e 78-12+1+3 = 69 != 61. Hence other option is to remove another digit altogether in addition to splitting.  An easy solution is:  Split 12 into 1 and 2 (now total 69),  and remove 8 (69-8 = 61).  If the crack splits 12 into half such that 1, 11, 10, 9 are on onside and the crack destroys 8 such that remaining (2, 1, 2, 3, 4, 5, 6, 7) are on other side the will add to 31, 30 and also smaller part totals to 31.



In [51]:
def find_clock_partitions(pn, parts):
    nparts = 0
    for (c1, c2) in parts:
        nparts += 1
        tc1, tc2 = sum(c1), sum(c2)
       
        if tc1 in (30, 31):
            lc1, lc2 = len(c1), len(c2)            
            if tc1+tc2 == 61 and ((tc1 == 30 and lc1 > lc2) or (tc1 == 31 and lc1 < lc2)):
                print(f"{pn}: {tc1} = {c1}  {tc2} = {c2}")

# clcok part sets after 8 is split into two zeros or destroyed, and one of 10, 11, or 12 is seperated into two numbers
clock_parts = {
'S8_10': [1, 2, 3, 4, 5, 6, 7, 0, 0, 9, 1, 0, 11, 12],
'S8_11' : [1, 2, 3, 4, 5, 6, 7, 0, 0, 9, 10, 1, 1, 12],
'S8_12' : [1, 2, 3, 4, 5, 6, 7, 0, 0, 9, 10, 11, 1, 2],    
}
    

for partition_name, p in clock_parts.items():
    print (partition_name, p)
    pretty_print(partition_name, generate_set_partitions(p, 2))

S8_10 [1, 2, 3, 4, 5, 6, 7, 0, 0, 9, 1, 0, 11, 12]
S8_10: 30 = [1, 3, 5, 0, 0, 9, 1, 0, 11]  31 = [2, 4, 6, 7, 12]
S8_10: 30 = [1, 2, 6, 0, 0, 9, 1, 0, 11]  31 = [3, 4, 5, 7, 12]
S8_10: 30 = [1, 2, 6, 0, 9, 1, 0, 11]  31 = [3, 4, 5, 7, 0, 12]
S8_10: 30 = [1, 3, 5, 0, 9, 1, 0, 11]  31 = [2, 4, 6, 7, 0, 12]
S8_10: 30 = [1, 2, 6, 0, 9, 1, 0, 11]  31 = [3, 4, 5, 7, 0, 12]
S8_10: 30 = [1, 3, 5, 0, 9, 1, 0, 11]  31 = [2, 4, 6, 7, 0, 12]
S8_10: 30 = [1, 4, 6, 7, 0, 1, 0, 11]  31 = [2, 3, 5, 0, 9, 12]
S8_10: 30 = [1, 2, 3, 5, 7, 0, 1, 0, 11]  31 = [4, 6, 0, 9, 12]
S8_10: 30 = [1, 2, 4, 5, 6, 0, 1, 0, 11]  31 = [3, 7, 0, 9, 12]
S8_10: 30 = [1, 2, 4, 5, 6, 1, 0, 11]  31 = [3, 7, 0, 0, 9, 12]
S8_10: 30 = [1, 2, 3, 5, 7, 1, 0, 11]  31 = [4, 6, 0, 0, 9, 12]
S8_10: 30 = [1, 4, 6, 7, 0, 1, 0, 11]  31 = [2, 3, 5, 0, 9, 12]
S8_10: 30 = [1, 2, 3, 5, 7, 0, 1, 0, 11]  31 = [4, 6, 0, 9, 12]
S8_10: 30 = [1, 2, 4, 5, 6, 0, 1, 0, 11]  31 = [3, 7, 0, 9, 12]
S8_10: 30 = [1, 2, 4, 5, 6, 0, 0, 1, 0, 11]  31 = [3,